In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))

    for c in ['MA_TP', 'BB_UPPER', 'BB_LOWER']:
        fig.add_trace(go.Scatter( 
            x=df_plot.time,
            y=df_plot[c],
            line=dict(width=2,color="#4290f5"),
            line_shape='spline',
            name=c,
            mode='lines'
            ))        

    df_markers = df_plot[df_plot.BB_SIGNAL!=0]
    fig.add_trace(go.Candlestick(
        x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#3480eb',
        decreasing_fillcolor="#3480eb",
        increasing_line_color='#3480eb',  
        decreasing_line_color='#3480eb'
    ))

    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [3]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [5]:
def Get_BB_Signal(row):
    if row.mid_c > row.BB_UPPER and row.mid_c > row.mid_o:
        return -1
    if row.mid_c < row.BB_LOWER and row.mid_c < row.mid_o:
        return 1
    return 0

In [6]:
df = df_raw[cols].copy()
df['TP'] = (df.mid_h + df.mid_l + df.mid_c) / 3
df['MA_TP'] = df.TP.rolling(window=20).mean()
df['STD_TP'] = df.TP.rolling(window=20).std()
df['BB_UPPER'] = df.MA_TP + 2 * df.STD_TP
df['BB_LOWER'] = df.MA_TP - 2 * df.STD_TP
df['BB_SIGNAL'] = df.apply(Get_BB_Signal, axis=1)

In [7]:
df_plot = df.iloc[1000:1150]
plot_candles(df_plot)

In [8]:
df['EMA_LONG'] = df.mid_c.ewm(span=26, min_periods=26).mean()
df['EMA_SHORT'] = df.mid_c.ewm(span=12, min_periods=12).mean()
df['MACD'] = df.EMA_SHORT - df.EMA_LONG
df['SIGNAL'] = df.MACD.ewm(span=9).mean()
df['HIST'] = df.MACD - df.SIGNAL
df.dropna(inplace=True)

In [9]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,TP,MA_TP,STD_TP,BB_UPPER,BB_LOWER,BB_SIGNAL,EMA_LONG,EMA_SHORT,MACD,SIGNAL,HIST
25,2023-01-06 02:00:00+00:00,133.804,134.378,133.584,134.163,134.041667,131.747183,1.316204,134.379592,129.114775,0,132.218055,132.855239,0.637184,0.637184,0.000000
26,2023-01-06 06:00:00+00:00,134.171,134.598,133.702,134.412,134.237333,131.936217,1.390498,134.717213,129.155220,0,132.403826,133.097403,0.693577,0.668513,0.025064
27,2023-01-06 10:00:00+00:00,134.414,134.778,133.235,133.372,133.795000,132.132000,1.360996,134.853993,129.410007,0,132.484945,133.140045,0.655100,0.663016,-0.007916
28,2023-01-06 14:00:00+00:00,133.370,134.102,132.082,132.112,132.765333,132.253617,1.299044,134.851705,129.655528,0,132.453998,132.980630,0.526631,0.616815,-0.090184
29,2023-01-06 18:00:00+00:00,132.108,132.214,131.994,132.094,132.100667,132.321717,1.250229,134.822175,129.821259,0,132.424389,132.843311,0.418921,0.557946,-0.139025


In [12]:
df_plot = df.iloc[-100:]
fig = go.Figure()

fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.SIGNAL,
    line=dict(width=2,color="#03adfc"),
    line_shape='spline',
    name='SIGNAL',
    mode='lines'
    ))        
fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.MACD,
    line=dict(width=2,color="#fc4a03"),
    line_shape='spline',
    name='MACD',
    mode='lines'
    ))         
fig.add_trace(go.Bar( 
    x=df_plot.time,
    y=df_plot.HIST,
    name='HIST'
    ))         

fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
    rangebreaks=[
        dict(bounds=["sat", "mon"])
    ]
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()